In [ ]:
# -*- coding: utf-8 -*-
"""
Notebook consolidado para otimização de preço de assinatura (churn) usando
Offline Reinforcement Learning (d3rlpy) e análise de risco (VaR/CVaR).

Este script combina:
- O dataset de Churn (RL_assinatura)
- A estrutura de simulação avançada e análise de risco (código_final_RL_OFF)
"""

# --- Etapa 1: Instalando e importando bibliotecas ---
print("--- Etapa 1: Instalando e importando bibliotecas ---")
%pip install d3rlpy --upgrade -q
%pip install kaggle -q
import torch
import pandas as pd
import numpy as np
import os
import shutil
import json
import matplotlib.pyplot as plt
from collections import defaultdict
import d3rlpy

# d3rlpy (Algoritmo e Q-Function)
from d3rlpy.algos import CQLConfig
from d3rlpy.models import QRQFunctionFactory # Para ações contínuas (do código final)

# d3rlpy (Dataset)
from d3rlpy.dataset import ReplayBuffer, FIFOBuffer, Episode
from d3rlpy.constants import ActionSpace

# Sklearn (ML Clássico e Métricas)
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print(f"d3rlpy versão: {d3rlpy.__version__}")
print(f"torch versão: {torch.__version__}")


# --- Etapa 2: Carregando e preparando os dados (do RL_assinatura) ---
print("\n--- Etapa 2: Carregando e preparando os dados ---")

# 2.1. Configuração da API do Kaggle
# ATENÇÃO: Faça upload do seu arquivo 'kaggle.json' para o /content/
# Este código irá movê-lo para o local correto.

kaggle_json_path = '/content/kaggle(1).json'
kaggle_dir = '/root/.kaggle(1)'
kaggle_dest_path = os.path.join(kaggle_dir, 'kaggle(1).json')

--- Etapa 1: Instalando e importando bibliotecas ---
d3rlpy versão: 2.8.1
torch versão: 2.8.0+cu126

--- Etapa 2: Carregando e preparando os dados ---


In [ ]:
print("\n--- Etapa 2: Carregando e preparando os dados ---")

# 2.1. Configuração da API do Kaggle (Lógica corrigida para 'kaggle (1).json')
# ATENÇÃO: Faça upload do seu 'kaggle.json' ou 'kaggle (1).json' para /content/
print("A procurar por 'kaggle (1).json' ou 'kaggle.json' em /content/...")

# Lista de nomes de ficheiro possíveis
possible_kaggle_files = ['/content/kaggle (1).json', '/content/kaggle.json']
kaggle_source_path = None

for f in possible_kaggle_files:
    if os.path.exists(f):
        kaggle_source_path = f
        print(f"Encontrado arquivo de credenciais: {f}")
        break

kaggle_dir = '/root/.kaggle'
kaggle_dest_path = os.path.join(kaggle_dir, 'kaggle.json')

if kaggle_source_path:
    try:
        os.makedirs(kaggle_dir, exist_ok=True)
        # Usar 'copy' é mais seguro para re-execuções
        shutil.copy(kaggle_source_path, kaggle_dest_path)
        os.chmod(kaggle_dest_path, 600)
        print(f"Credenciais copiadas para {kaggle_dest_path} e permissões definidas.")
    except Exception as e:
        print(f"Erro ao mover/copiar o ficheiro kaggle: {e}")
else:
    print("Aviso: Nenhum 'kaggle.json' ou 'kaggle (1).json' encontrado em /content/.")
    print("Por favor, faça o upload do seu ficheiro de credenciais e rode esta célula novamente.")


--- Etapa 2: Carregando e preparando os dados ---
A procurar por 'kaggle (1).json' ou 'kaggle.json' em /content/...
Encontrado arquivo de credenciais: /content/kaggle.json
Credenciais copiadas para /root/.kaggle/kaggle.json e permissões definidas.


In [ ]:
# 2.2. Baixar e carregar o dataset
# (Usando o dataset do seu notebook RL_assinatura)
csv_file_path = 'Subscription_Service_Churn_Dataset.csv'

# Limpa downloads anteriores para forçar um novo download caso o unzip tenha falhado
if os.path.exists('subscription-churn-dataset.zip'):
    os.remove('subscription-churn-dataset.zip')
if os.path.exists(csv_file_path):
    os.remove(csv_file_path)

try:
    print("A tentar baixar o dataset 'sameerhussain007/subscription-churn-dataset'...")
    !kaggle datasets download -d sameerhussain007/subscription-churn-dataset
    !unzip -o subscription-churn-dataset.zip

    df_original = pd.read_csv(csv_file_path)
    print(f"\nSUCESSO! Dataset '{csv_file_path}' carregado.")
    print(f"Dataset original com {len(df_original)} linhas.")

except Exception as e:
    print(f"\nOcorreu um erro ao baixar ou carregar o dataset: {e}")
    print("Verifique se as suas credenciais 'kaggle.json' são válidas.")
    df_original = pd.DataFrame()

A tentar baixar o dataset 'sameerhussain007/subscription-churn-dataset'...
Dataset URL: https://www.kaggle.com/datasets/sameerhussain007/subscription-churn-dataset
License(s): CC0-1.0
  0% 0.00/63.3k [00:00<?, ?B/s]
100% 63.3k/63.3k [00:00<00:00, 110MB/s]
Archive:  subscription-churn-dataset.zip
  inflating: Subscription_Service_Churn_Dataset.csv  

SUCESSO! Dataset 'Subscription_Service_Churn_Dataset.csv' carregado.
Dataset original com 963 linhas.


In [ ]:
# 2.3. Limpeza inicial e tratamento de NaNs
if not df_original.empty:
    # Remover colunas de ID
    df_original = df_original.drop('CustomerID', axis=1, errors='ignore')

    # Identificar colunas numéricas e categóricas para tratamento
    numeric_cols = ['AccountAge', 'MonthlyCharges', 'TotalCharges',
                    'ViewingHoursPerWeek', 'AverageViewingDuration',
                    'ContentDownloadsPerMonth', 'UserRating',
                    'SupportTicketsPerMonth', 'WatchlistSize']

    category_cols = ['SubscriptionType', 'PaymentMethod', 'PaperlessBilling',
                     'ContentType', 'MultiDeviceAccess', 'DeviceRegistered',
                     'GenrePreference', 'Gender', 'ParentalControl', 'SubtitlesEnabled']

    # Tratar NaNs
    for col in numeric_cols:
        if col in df_original.columns:
            median_val = df_original[col].median()
            df_original[col].fillna(median_val, inplace=True)

    for col in category_cols:
        if col in df_original.columns:
            df_original[col].fillna('Desconhecido', inplace=True)

    # Garante que 'Churn' é numérico
    if 'Churn' in df_original.columns:
         df_original['Churn'] = pd.to_numeric(df_original['Churn'], errors='coerce').fillna(0)
    else:
        print("ERRO: Coluna 'Churn' não encontrada. A simulação irá falhar.")

    print("Limpeza inicial e tratamento de NaNs concluídos.")
    print(df_original.head())
else:
    print("PULANDO Etapa 2.3: Dataset original está vazio ou não foi carregado.")

Limpeza inicial e tratamento de NaNs concluídos.
   AccountAge  MonthlyCharges  TotalCharges SubscriptionType  \
0          42       11.321950    475.521914            Basic   
1          95       12.810915   1217.036887         Standard   
2           6       12.169888     91.583304         Standard   
3          54       17.917819    967.562224            Basic   
4          27       12.169888    339.057244            Basic   

      PaymentMethod PaperlessBilling ContentType MultiDeviceAccess  \
0  Electronic check              Yes      Movies               Yes   
1  Electronic check              Yes    TV Shows                No   
2       Credit card              Yes    TV Shows                No   
3      Desconhecido              Yes      Movies                No   
4      Mailed check               No    TV Shows                No   

  DeviceRegistered  ViewingHoursPerWeek  AverageViewingDuration  \
0           Tablet             0.386852               24.593361   
1          

/tmp/ipython-input-1957457221.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_original[col].fillna(median_val, inplace=True)
/tmp/ipython-input-1957457221.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [ ]:
# --- Etapa 3: Geração do ambiente sintético para RL (Adaptado) ---
print("\n--- Etapa 3: Gerando o ambiente sintético para o treinamento ---")

if not df_original.empty:
    # 3.1. Calcular métricas base (adaptado para o dataset de churn)
    print("Calculando métricas base (Taxa de Retenção, Preço Médio) por 'SubscriptionType'...")

    group_col = 'SubscriptionType'
    if group_col not in df_original.columns:
        print(f"Aviso: Coluna '{group_col}' não encontrada. Usando 'Basic' como fallback.")
        df_original[group_col] = 'Basic'

    # 1. Taxa de Retenção Base (1 - Churn Rate)
    base_retention_metrics = (1.0 - df_original.groupby(group_col)['Churn'].mean()).to_dict()

    # 2. Preço de Referência (Média do MonthlyCharges)
    reference_price_metrics = df_original.groupby(group_col)['MonthlyCharges'].mean().to_dict()

    # 3. Fallbacks
    global_base_retention = 1.0 - df_original['Churn'].mean()
    global_reference_price = df_original['MonthlyCharges'].mean()
    # 3.2. Definir Tiers e Elasticidade
    product_tiers = {}
    unique_tiers = df_original[group_col].unique()
    if len(unique_tiers) == 0: unique_tiers = ['Basic'] # Segurança

    for tier_name in unique_tiers:
        # Garantir que o nome do tier é uma string válida (ex: lida como NaN)
        tier_key = str(tier_name) if pd.notna(tier_name) else 'Desconhecido'

        base_ret = base_retention_metrics.get(tier_key, global_base_retention)
        ref_price = reference_price_metrics.get(tier_key, global_reference_price)

        product_tiers[tier_key] = {
            "base_retention": base_ret,
            "reference_price": ref_price,
            "min_price": max(5.0, ref_price * 0.5),
            "max_price": max(10.0, ref_price * 1.5), # Garante max > min
        }

    # Se 'Desconhecido' não foi pego no loop, mas existe, adiciona
    if 'Desconhecido' not in product_tiers and 'Desconhecido' in base_retention_metrics:
        ref_price = reference_price_metrics.get('Desconhecido', global_reference_price)
        product_tiers['Desconhecido'] = {
            "base_retention": base_retention_metrics.get('Desconhecido', global_base_retention),
            "reference_price": ref_price,
            "min_price": max(5.0, ref_price * 0.5),
            "max_price": max(10.0, ref_price * 1.5),
        }

    ELASTICITY = 0.015 # Parâmetro de simulação
    N_SAMPLES_PER_ROW = 50 # Amostras de preço por cliente

    print("Dicionário de 'Tiers' (product_tiers) criado:")
    print(json.dumps(product_tiers, indent=2))
    # 3.3. Geração dos Dados Sintéticos (Loop Principal)
    print("Iniciando loop de geração de dados sintéticos...")
    generated_data = []

    for index, row in df_original.iterrows():
        state_context = row.to_dict()

        # O tier do cliente é o seu tipo de assinatura
        client_tier_name = str(state_context.get(group_col, 'Basic'))
        if pd.isna(client_tier_name):
             client_tier_name = 'Desconhecido'

        # Pega as propriedades desse tier
        if client_tier_name not in product_tiers:
            # Fallback muito genérico se algo falhar
            tier_props = {"base_retention": global_base_retention,
                          "reference_price": global_reference_price,
                          "min_price": max(5.0, global_reference_price * 0.5),
                          "max_price": max(10.0, global_reference_price * 1.5)}
        else:
            tier_props = product_tiers[client_tier_name]

        base_retention = tier_props["base_retention"]
        price_reference = tier_props["reference_price"]

        for _ in range(N_SAMPLES_PER_ROW):
            price_action = np.random.uniform(tier_props["min_price"], tier_props["max_price"])

            # Simulação da Retenção (Dinâmica do Ambiente)
            simulated_retention_rate = base_retention * np.exp(-ELASTICITY * (price_action - price_reference))
            simulated_retention_rate = max(0, min(1, simulated_retention_rate))

            # Recompensa (Reward) = Lucro esperado
            simulated_profit_reward = simulated_retention_rate * price_action

            # Adicionar dados gerados à lista
            new_row = state_context.copy()
            new_row['Price_Action'] = price_action
            new_row['Simulated_Profit_Reward'] = simulated_profit_reward
            generated_data.append(new_row)

    print("Loop concluído.")

    # 3.4. Criação do DataFrame Final
    df_rl_training = pd.DataFrame(generated_data)
    df_rl_training.dropna(subset=['Simulated_Profit_Reward'], inplace=True)

    print(f"Dataset sintético para treino gerado com {len(df_rl_training)} linhas.")
    if not df_rl_training.empty:
        print(df_rl_training[['SubscriptionType', 'Price_Action', 'Simulated_Profit_Reward']].head())
    else:
        print("ERRO: O dataset sintético está vazio após a geração.")

else:
    print("PULANDO Etapa 3: Dataset original está vazio.")
    df_rl_training = pd.DataFrame()


--- Etapa 3: Gerando o ambiente sintético para o treinamento ---
Calculando métricas base (Taxa de Retenção, Preço Médio) por 'SubscriptionType'...
Dicionário de 'Tiers' (product_tiers) criado:
{
  "Basic": {
    "base_retention": 0.8122866894197952,
    "reference_price": 12.052983185163434,
    "min_price": 6.026491592581717,
    "max_price": 18.07947477774515
  },
  "Standard": {
    "base_retention": 0.8362369337979094,
    "reference_price": 12.428250080221844,
    "min_price": 6.214125040110922,
    "max_price": 18.642375120332765
  },
  "Premium": {
    "base_retention": 0.8318318318318318,
    "reference_price": 12.338196909434597,
    "min_price": 6.1690984547172985,
    "max_price": 18.507295364151894
  },
  "Desconhecido": {
    "base_retention": 0.78,
    "reference_price": 12.655128547991218,
    "min_price": 6.327564273995609,
    "max_price": 18.982692821986827
  }
}
Iniciando loop de geração de dados sintéticos...
Loop concluído.
Dataset sintético para treino gerado co

In [ ]:
# --- Etapa 3.A: Dividindo o Dataset Sintético em Treino e Teste ---
print("\n--- Etapa 3.A: Dividindo o Dataset Sintético em Treino e Teste ---")

if not df_rl_training.empty:
    df_train_rl, df_test_rl = train_test_split(df_rl_training, test_size=0.2, random_state=42)
    df_train_rl = df_train_rl.reset_index(drop=True)
    df_test_rl = df_test_rl.reset_index(drop=True)

    print(f"Dataset sintético dividido:")
    print(f"  - Treino: {len(df_train_rl)} amostras")
    print(f"  - Teste:  {len(df_test_rl)} amostras")
else:
    print("PULANDO Etapa 3.A: 'df_rl_training' está vazio.")
    df_train_rl, df_test_rl = pd.DataFrame(), pd.DataFrame()


--- Etapa 3.A: Dividindo o Dataset Sintético em Treino e Teste ---
Dataset sintético dividido:
  - Treino: 38520 amostras
  - Teste:  9630 amostras


In [ ]:
# --- Etapa 4: Formatando dados de TREINO e TESTE para o d3rlpy (CORRIGIDO v2) ---
print("\n--- Etapa 4: Formatando dados de TREINO e TESTE para o d3rlpy ---")

# Inicializa as variáveis globais fora do IF-block
state_features = []
category_state_features = []
observation_cols_map = []
train_replay_buffer = None
test_replay_buffer = None
train_buffer_creation_successful = False
test_buffer_creation_successful = False

if not df_train_rl.empty and not df_original.empty:

    # 4.1. Definir Features de Estado
    all_original_cols = set(df_original.columns)
    cols_to_exclude = {'Churn', 'MonthlyCharges', 'TotalCharges'}
    state_features = sorted(list(all_original_cols - cols_to_exclude))
    category_state_features = sorted(list(set(category_cols) & set(state_features)))

    print(f"Features de Estado (State) identificadas ({len(state_features)}): {state_features}")
    print(f"Features Categóricas (para One-Hot) ({len(category_state_features)}): {category_state_features}")

    # 4.2. Função para processar DataFrame (Treino ou Teste)
    # --- CORREÇÃO AQUI: Adicionado 'is_inference=False' ---
    def process_df_for_d3rlpy(df, state_cols, category_cols_in_state, observation_cols_map=None, is_inference=False):
        """Converte um DataFrame para o formato de arrays do d3rlpy."""

        df_processed = df.copy()

        # 1. One-Hot Encoding
        df_onehot = pd.get_dummies(df_processed[state_cols],
                                   columns=category_cols_in_state,
                                   dummy_na=False)

        # 2. Alinhamento de Colunas
        if observation_cols_map is None:
            observation_cols_map = sorted(list(df_onehot.columns))
        else:
            df_onehot = df_onehot.reindex(columns=observation_cols_map, fill_value=0)

        # 3. Criar Arrays NumPy
        observations = df_onehot[observation_cols_map].values.astype(np.float32)

        # --- CORREÇÃO AQUI: Só processa ações/recompensas se NÃO for inferência ---
        if not is_inference:
            actions = df_processed[['Price_Action']].values.astype(np.float32)
            rewards = df_processed['Simulated_Profit_Reward'].values.astype(np.float32).reshape(-1, 1)
            terminated = np.ones_like(rewards, dtype=np.float32).reshape(-1, 1)
            return observations, actions, rewards, terminated, observation_cols_map
        else:
            # Se for inferência, só precisamos das observações
            return observations, None, None, None, observation_cols_map
    # -----------------------------------------------------------------

    # 4.3. Função para criar ReplayBuffer
    def create_replay_buffer(observations, actions, rewards, terminated):
        """Cria um ReplayBuffer do d3rlpy a partir de arrays NumPy."""
        try:
            buffer_size = len(rewards)
            episodes = []
            for i in range(buffer_size):
                episode = Episode(
                    observations=observations[i:i+1],
                    actions=actions[i:i+1],
                    rewards=rewards[i:i+1],
                    terminated=terminated[i:i+1]
                )
                episodes.append(episode)

            replay_buffer = ReplayBuffer(
                buffer=FIFOBuffer(limit=buffer_size),
                episodes=episodes,
                cache_size=16
            )
            return replay_buffer
        except Exception as e:
            print(f"!!! ERRO CRÍTICO ao criar Buffer: {e}")
            return None

    # 4.4. Processar TREINO (is_inference continua False por defeito)
    print("\n--- Processando Conjunto de Treino ---")
    train_obs, train_act, train_rew, train_term, observation_cols_map = \
        process_df_for_d3rlpy(df_train_rl, state_features, category_state_features, observation_cols_map=None)

    print(f"Número de colunas de observação (features) após one-hot: {len(observation_cols_map)}")

    train_replay_buffer = create_replay_buffer(train_obs, train_act, train_rew, train_term)
    if train_replay_buffer:
        print(f"SUCESSO: ReplayBuffer de TREINO pronto com {len(train_replay_buffer.buffer)} transições.")
        train_buffer_creation_successful = True

    # 4.5. Processar TESTE (is_inference continua False por defeito)
    print("\n--- Processando Conjunto de Teste ---")
    test_obs, test_act, test_rew, test_term, _ = \
        process_df_for_d3rlpy(df_test_rl, state_features, category_state_features, observation_cols_map=observation_cols_map)

    test_replay_buffer = create_replay_buffer(test_obs, test_act, test_rew, test_term)
    if test_replay_buffer:
        print(f"SUCESSO: ReplayBuffer de TESTE pronto com {len(test_replay_buffer.buffer)} transições.")
        test_buffer_creation_successful = True

else:
    print("PULANDO Etapa 4: Datasets de treino/teste estão vazios.")


--- Etapa 4: Formatando dados de TREINO e TESTE para o d3rlpy ---
Features de Estado (State) identificadas (17): ['AccountAge', 'AverageViewingDuration', 'ContentDownloadsPerMonth', 'ContentType', 'DeviceRegistered', 'Gender', 'GenrePreference', 'MultiDeviceAccess', 'PaperlessBilling', 'ParentalControl', 'PaymentMethod', 'SubscriptionType', 'SubtitlesEnabled', 'SupportTicketsPerMonth', 'UserRating', 'ViewingHoursPerWeek', 'WatchlistSize']
Features Categóricas (para One-Hot) (10): ['ContentType', 'DeviceRegistered', 'Gender', 'GenrePreference', 'MultiDeviceAccess', 'PaperlessBilling', 'ParentalControl', 'PaymentMethod', 'SubscriptionType', 'SubtitlesEnabled']

--- Processando Conjunto de Treino ---
Número de colunas de observação (features) após one-hot: 42
2025-11-05 16:27.21 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(42,)]) reward_signat

In [ ]:
# --- Etapa 5: Configurando, Construindo e Treinando o Agente (CORRIGIDO v4) ---
print("\n--- Etapa 5: Configurando, Construindo e Treinando o Agente ---")

model_built_successfully = False
agent_trained_successfully = False
cql_pricer = None # O nome do agente

if train_buffer_creation_successful:
    try:
        print("Configurando o agente CQL com QRQFunctionFactory (para Ações Contínuas)...")

        # 5.1. Configurar o Agente
        cql_config = CQLConfig(
            q_func_factory=QRQFunctionFactory(n_quantiles=64),
            batch_size=256,
            n_action_samples=10,
            alpha_learning_rate=1e-4,
            conservative_weight=5.0
        )

        # 5.2. Criar o Agente
        device_to_use_str = "cuda" if torch.cuda.is_available() else "cpu"
        cql_pricer = cql_config.create(device=device_to_use_str)
        print(f"Agente criado e rodando em: {cql_pricer._device}")

        # 5.3. Construir o Agente com os dados
        print("Construindo o agente com as assinaturas do ReplayBuffer de TREINO...")
        cql_pricer.build_with_dataset(train_replay_buffer)
        model_built_successfully = True
        print("Agente construído com sucesso.")

        # 5.4. Treinar o Agente (Offline)
        N_TRAINING_EPOCHS = 10
        N_STEPS_PER_EPOCH = 100

        print(f"Iniciando treinamento offline por {N_TRAINING_EPOCHS} épocas ({N_STEPS_PER_EPOCH} steps/epoch)...")

        # --- CORREÇÃO AQUI ---
        # Removidos 'scorers' e 'eval_dataset' para evitar o TypeError,
        # alinhando com o notebook de referência (código_final_RL_OFF).

        cql_pricer.fit(
            train_replay_buffer,
            n_steps=N_TRAINING_EPOCHS * N_STEPS_PER_EPOCH,
            n_steps_per_epoch=N_STEPS_PER_EPOCH
        )

        agent_trained_successfully = True
        print("\n--- Treinamento Concluído com Sucesso ---")

    except Exception as e:
        print(f"\n!!! ERRO CRÍTICO durante a Etapa 5 (Construção/Treinamento): {e} !!!")
        import traceback
        traceback.print_exc()

else:
    print("Aviso: Treinamento PULADO. 'train_replay_buffer' não foi criado.")


--- Etapa 5: Configurando, Construindo e Treinando o Agente ---
Configurando o agente CQL com QRQFunctionFactory (para Ações Contínuas)...
Agente criado e rodando em: cpu
Construindo o agente com as assinaturas do ReplayBuffer de TREINO...
Agente construído com sucesso.
Iniciando treinamento offline por 10 épocas (100 steps/epoch)...
2025-11-05 16:27.34 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float32')], shape=[(42,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=1)
2025-11-05 16:27.34 [warning  ] Skip building models since they're already built.
2025-11-05 16:27.34 [info     ] Directory is created at d3rlpy_logs/CQL_20251105162734
2025-11-05 16:27.34 [info     ] Parameters                     params={'observation_shape': [42], 'action_size': 1, 'config': {'type': 'cql

Epoch 1/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-11-05 16:28.09 [info     ] CQL_20251105162734: epoch=1 step=100 epoch=1 metrics={'time_sample_batch': 0.008755757808685302, 'time_algorithm_update': 0.3387613415718079, 'critic_loss': 87.24948081970214, 'conservative_loss': -70.18566291809083, 'alpha': 0.994828377366066, 'actor_loss': -7.992976068854332, 'temp': 0.9967465716600418, 'temp_loss': 1.408757402896881, 'time_step': 0.3476544260978699} step=100
2025-11-05 16:28.09 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20251105162734/model_100.d3


Epoch 2/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-11-05 16:28.43 [info     ] CQL_20251105162734: epoch=2 step=200 epoch=2 metrics={'time_sample_batch': 0.009012703895568847, 'time_algorithm_update': 0.3325561237335205, 'critic_loss': -135.73632007598877, 'conservative_loss': -151.86371910095215, 'alpha': 0.9821200197935105, 'actor_loss': -1.0730998655594886, 'temp': 0.986779014468193, 'temp_loss': 1.5636797916889191, 'time_step': 0.3417008757591248} step=200
2025-11-05 16:28.43 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20251105162734/model_200.d3


...

2025-11-05 16:34.05 [info     ] CQL_20251105162734: epoch=8 step=800 epoch=8 metrics={'time_sample_batch': 0.009103341102600098, 'time_algorithm_update': 0.9102189230918885, 'critic_loss': -6006.136733398437, 'conservative_loss': -7691.232084960938, 'alpha': 0.8662542647123337, 'actor_loss': 850.3849615478516, 'temp': 1.0002096778154372, 'temp_loss': -2.9471511054039, 'time_step': 0.919463529586792} step=800
2025-11-05 16:34.05 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20251105162734/model_800.d3


Epoch 9/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-11-05 16:35.19 [info     ] CQL_20251105162734: epoch=9 step=900 epoch=9 metrics={'time_sample_batch': 0.009109461307525634, 'time_algorithm_update': 0.7249660420417786, 'critic_loss': -10101.297177734376, 'conservative_loss': -12957.51720703125, 'alpha': 0.8443763309717178, 'actor_loss': 1472.6126647949218, 'temp': 1.0179425823688506, 'temp_loss': -3.58377925157547, 'time_step': 0.734211950302124} step=900
2025-11-05 16:35.19 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20251105162734/model_900.d3


Epoch 10/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-11-05 16:36.22 [info     ] CQL_20251105162734: epoch=10 step=1000 epoch=10 metrics={'time_sample_batch': 0.009180173873901368, 'time_algorithm_update': 0.6224578094482421, 'critic_loss': -15580.564853515625, 'conservative_loss': -20202.903984375, 'alpha': 0.8236886262893677, 'actor_loss': 2343.871702880859, 'temp': 1.0354898595809936, 'temp_loss': -4.040697162151337, 'time_step': 0.6317691874504089} step=1000
2025-11-05 16:36.22 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20251105162734/model_1000.d3

--- Treinamento Concluído com Sucesso ---


In [ ]:
# --- Sub-Etapa 5.5: Definição das Funções para Análise de Risco ---
print("\n--- Sub-Etapa 5.5: Definindo funções para análise de risco (VaR, CVaR) ---")

def get_quantile_values(agent, observation_np, action_np):
    """Obtém os valores dos quantis previstos pela Q-Function (QR) do agente."""
    if not (agent and model_built_successfully and agent_trained_successfully):
        print("[Debug get_quantile_values]: Agente não está pronto.")
        return None

    try:
        device = agent._device
        obs_tensor = torch.tensor(observation_np, dtype=torch.float32, device=device).reshape(1, -1)
        act_tensor = torch.tensor(action_np, dtype=torch.float32, device=device).reshape(1, -1)

        if not (hasattr(agent, 'impl') and hasattr(agent.impl, '_q_func_forwarder') and
                agent.impl._q_func_forwarder._forwarders):
            print("[Debug get_quantile_values]: Estrutura do agente (impl._q_func_forwarder) não encontrada.")
            return None

        q_func_forwarder = agent.impl._q_func_forwarder._forwarders[0]
        critic_network = q_func_forwarder._q_func
        critic_network.eval() # Modo de avaliação

        with torch.no_grad():
            q_output = critic_network(obs_tensor, act_tensor)

        quantile_tensor = q_output.quantiles
        quantile_values_np = quantile_tensor.cpu().numpy().squeeze()

        if isinstance(quantile_values_np, np.ndarray) and quantile_values_np.ndim == 1:
             return quantile_values_np
        else:
             print("[Debug get_quantile_values]: A saída dos quantis não é um array 1D.")
             return None

    except Exception as e:
        print(f"!!! Erro inesperado em get_quantile_values: {e} !!!")
        import traceback
        traceback.print_exc()
        return None

def calculate_var(distribution_values, alpha=0.05):
    """Calcula o Valor em Risco (VaR)."""
    if distribution_values is None or not isinstance(distribution_values, np.ndarray) or distribution_values.size == 0:
        return np.nan
    sorted_returns = np.sort(distribution_values)
    var_index = int(alpha * len(sorted_returns))
    var_index = max(0, min(var_index, len(sorted_returns) - 1))
    return sorted_returns[var_index]

def calculate_cvar(distribution_values, alpha=0.05):
    """Calcula o Valor Condicional em Risco (CVaR)."""
    if distribution_values is None or not isinstance(distribution_values, np.ndarray) or distribution_values.size == 0:
        return np.nan
    var_value = calculate_var(distribution_values, alpha)
    if np.isnan(var_value): return np.nan
    worse_than_var = distribution_values[distribution_values <= var_value]
    return np.mean(worse_than_var) if worse_than_var.size > 0 else np.nan

print("Funções de análise de risco definidas.")


--- Sub-Etapa 5.5: Definindo funções para análise de risco (VaR, CVaR) ---
Funções de análise de risco definidas.


In [ ]:
# --- Sub-Etapa 5.6: Avaliação de Risco da Política (em dados de TESTE) ---
print("\n--- Sub-Etapa 5.6: Avaliação de Risco da Política (em dados de TESTE) ---")

if agent_trained_successfully and test_buffer_creation_successful:
    SAMPLE_SIZE = min(1000, len(test_replay_buffer.buffer))
    ALPHA_RISK = 0.05

    sample_indices = np.random.choice(len(test_replay_buffer.buffer), SAMPLE_SIZE, replace=False)

    results = []
    actual_rewards_eval = []
    predicted_mean_profits_eval = []

    print(f"Processando {SAMPLE_SIZE} transições aleatórias do ReplayBuffer de TESTE...")

    for index in sample_indices:
        try:
            transition = test_replay_buffer.episodes[index]
            obs = np.asarray(transition.observations[0])
            actual_reward = np.asarray(transition.rewards[0])[0]

            # Prever Ação Ótima
            optimal_action = cql_pricer.predict(obs.reshape(1, -1))[0]

            # Obter Distribuição de Quantis
            predicted_quantiles = get_quantile_values(cql_pricer, obs, optimal_action)

            if predicted_quantiles is not None and predicted_quantiles.size > 0:
                mean_profit = np.mean(predicted_quantiles)
                var_value = calculate_var(predicted_quantiles, alpha=ALPHA_RISK)
                cvar_value = calculate_cvar(predicted_quantiles, alpha=ALPHA_RISK)

                results.append({
                    'predicted_price': optimal_action[0],
                    'mean_sim_profit_agent': mean_profit,
                    'VaR_5': var_value,
                    'CVaR_5': cvar_value
                })

                actual_rewards_eval.append(actual_reward)
                predicted_mean_profits_eval.append(mean_profit)
        except Exception as e:
            print(f"Erro ao processar amostra {index}: {e}")

    # 5.7. Apresentar Resultados
    if results:
        results_df = pd.DataFrame(results)

        print(f"\n--- Resultados Agregados da Avaliação de Risco (Buffer de TESTE) ---")
        print(f" Nível Alpha: {ALPHA_RISK*100:.1f}% | Amostras: {len(results_df)}")
        print("-" * 70)
        print(f" Preço Médio Recomendado pela Política:        ${results_df['predicted_price'].mean():,.2f}")
        print(f" Lucro Simulado Médio PREVISTO pelo Agente:    ${results_df['mean_sim_profit_agent'].mean():,.2f}")
        print(f" VaR (5%) Médio Simulado (Previsto Agente): ${results_df['VaR_5'].mean(skipna=True):,.2f}")
        print(f" CVaR (5%) Médio Simulado (Previsto Agente):${results_df['CVaR_5'].mean(skipna=True):,.2f}")
        print("-" * 70)

        if actual_rewards_eval:
            mae_eval = mean_absolute_error(actual_rewards_eval, predicted_mean_profits_eval)
            avg_actual_profit_buffer = np.mean(actual_rewards_eval)
            print(f"\n--- Métricas de PRECISÃO da Previsão Média do Agente (vs Buffer TESTE) ---")
            print(f" Lucro Simulado Médio REAL no Buffer Avaliado: ${avg_actual_profit_buffer:,.2f}")
            print(f" MAE (Erro Médio Absoluto da Previsão Média):  ${mae_eval:,.2f}")
            print("-" * 70)
    else:
        print("Nenhum resultado de avaliação de risco foi gerado.")

else:
    print("Avaliação de Risco PULADA. Agente ou buffer de teste não estão prontos.")


--- Sub-Etapa 5.6: Avaliação de Risco da Política (em dados de TESTE) ---
Processando 1000 transições aleatórias do ReplayBuffer de TESTE...

--- Resultados Agregados da Avaliação de Risco (Buffer de TESTE) ---
 Nível Alpha: 5.0% | Amostras: 1000
----------------------------------------------------------------------
 Preço Médio Recomendado pela Política:        $1.00
 Lucro Simulado Médio PREVISTO pelo Agente:    $-2,649.53
 VaR (5%) Médio Simulado (Previsto Agente): $-7,438.05
 CVaR (5%) Médio Simulado (Previsto Agente):$-8,913.42
----------------------------------------------------------------------

--- Métricas de PRECISÃO da Previsão Média do Agente (vs Buffer TESTE) ---
 Lucro Simulado Médio REAL no Buffer Avaliado: $10.12
 MAE (Erro Médio Absoluto da Previsão Média):  $2,659.65
----------------------------------------------------------------------


In [ ]:
# --- Etapa 6: Gerando Recomendações de Preço Específicas (CORRIGIDO) ---
print("\n--- Etapa 6: Gerando Recomendações de Preço Específicas ---")

# Variáveis globais necessárias (da Etapa 4)
# state_features, observation_cols_map, category_state_features

def get_price_recommendation(**scenario_kwargs):
    """
    Gera uma recomendação de preço para um cenário de cliente.
    As kwargs devem corresponder às 'state_features' do dataset de churn.
    """

    if not (agent_trained_successfully and 'observation_cols_map' in globals() and observation_cols_map):
        print("Erro: Agente não treinado ou 'observation_cols_map' não definido.")
        return

    # 1. Criar DataFrame do cenário
    default_scenario = {}
    for col in state_features:
        if col in df_original.columns:
            if df_original[col].dtype == 'object':
                default_scenario[col] = df_original[col].mode()[0]
            else:
                default_scenario[col] = df_original[col].median()
        else:
            default_scenario[col] = 0

    for key, value in scenario_kwargs.items():
        if key in default_scenario:
            default_scenario[key] = value

    scenario_df = pd.DataFrame([default_scenario])

    # 2. Processar o cenário (One-hot e Alinhamento)
    try:
        # --- CORREÇÃO AQUI: Passa 'is_inference=True' ---
        scenario_obs, _, _, _, _ = process_df_for_d3rlpy(
            scenario_df,
            state_features,
            category_state_features,
            observation_cols_map=observation_cols_map,
            is_inference=True  # Diz à função para não procurar 'Price_Action'
        )
        observation = scenario_obs.reshape(1, -1)

    except Exception as e:
        print(f"Erro ao processar o cenário: {e}")
        return

    # 3. Prever Ação (Preço) e Risco
    try:
        recommended_price = cql_pricer.predict(observation)[0]

        print(f"\nCenário:")
        print(json.dumps(scenario_kwargs, indent=2))
        print(f"  => Preço Recomendado: ${recommended_price[0]:.2f}")

        # 4. Calcular Risco (VaR/CVaR)
        predicted_quantiles = get_quantile_values(cql_pricer, observation, recommended_price)

        if predicted_quantiles is not None and predicted_quantiles.size > 0:
            mean_profit = np.mean(predicted_quantiles)
            var_5 = calculate_var(predicted_quantiles, alpha=0.05)
            cvar_5 = calculate_cvar(predicted_quantiles, alpha=0.05)
            print(f"     Lucro Médio Previsto: ${mean_profit:.2f}")
            print(f"     VaR (5%): ${var_5:.2f} | CVaR (5%): ${cvar_5:.2f}")
        else:
            print("     (Não foi possível calcular VaR/CVaR para esta recomendação)")
        print("-" * 30)

    except Exception as e:
        print(f"Erro durante a predição para o cenário: {e}")

# --- Exemplos de Recomendação ---
if agent_trained_successfully:
    print("Gerando recomendações de exemplo...")
    get_price_recommendation(
        SubscriptionType='Standard',
        Gender='Female',
        DeviceRegistered='Tablet',
        ViewingHoursPerWeek=40
    )

    get_price_recommendation(
        SubscriptionType='Basic',
        Gender='Male',
        DeviceRegistered='Mobile',
        AccountAge=5 # Baixa idade
    )

    get_price_recommendation(
        SubscriptionType='Premium',
        ViewingHoursPerWeek=50,
        ContentDownloadsPerMonth=30,
        UserRating=4.5
    )
else:
    print("Recomendações PULADAS. Agente não treinado.")


--- Etapa 6: Gerando Recomendações de Preço Específicas ---
Gerando recomendações de exemplo...

Cenário:
{
  "SubscriptionType": "Standard",
  "Gender": "Female",
  "DeviceRegistered": "Tablet",
  "ViewingHoursPerWeek": 40
}
  => Preço Recomendado: $1.00
     Lucro Médio Previsto: $-2941.21
     VaR (5%): $-8269.86 | CVaR (5%): $-9912.32
------------------------------

Cenário:
{
  "SubscriptionType": "Basic",
  "Gender": "Male",
  "DeviceRegistered": "Mobile",
  "AccountAge": 5
}
  => Preço Recomendado: $1.00
     Lucro Médio Previsto: $-2394.98
     VaR (5%): $-6443.26 | CVaR (5%): $-7693.88
------------------------------

Cenário:
{
  "SubscriptionType": "Premium",
  "ViewingHoursPerWeek": 50,
  "ContentDownloadsPerMonth": 30,
  "UserRating": 4.5
}
  => Preço Recomendado: $1.00
     Lucro Médio Previsto: $-3246.75
     VaR (5%): $-9107.43 | CVaR (5%): $-10914.38
------------------------------


In [ ]:
# --- Etapa 9: Comparação com Aprendizado Supervisionado (SL-Regressor) ---
print("\n--- Etapa 9: Iniciando Comparação com Supervised Learning (LGBM) ---")

if not df_rl_training.empty and 'observation_cols_map' in globals():

    print("Formatando dados para o modelo de SL...")

    # Re-processa o df_train_rl
    df_train_sl_X, _, _, _, _ = process_df_for_d3rlpy(
        df_train_rl, state_features, category_state_features, observation_cols_map
    )
    df_train_sl_X = pd.DataFrame(df_train_sl_X, columns=observation_cols_map)
    df_train_sl_X['Price_Action'] = df_train_rl['Price_Action']

    y_train_sl = df_train_rl['Simulated_Profit_Reward']

    # Repete para o Teste
    df_test_sl_X, _, _, _, _ = process_df_for_d3rlpy(
        df_test_rl, state_features, category_state_features, observation_cols_map
    )
    df_test_sl_X = pd.DataFrame(df_test_sl_X, columns=observation_cols_map)
    df_test_sl_X['Price_Action'] = df_test_rl['Price_Action']

    y_test_sl = df_test_rl['Simulated_Profit_Reward']

    print(f"Dados SL divididos em {len(df_train_sl_X)} para treino e {len(df_test_sl_X)} para teste.")

    # 9.2. Treinar o Regressor LightGBM
    print("\nTreinando o modelo LightGBM para prever o lucro...")

    lgbm_regressor = lgb.LGBMRegressor(random_state=42, n_estimators=200)
    lgbm_regressor.fit(df_train_sl_X, y_train_sl)
    print("Modelo SL (Regressor) treinado com sucesso!")

    # 9.3. Avaliar a PRECISÃO do Regressor
    print("\nAvaliando a precisão do modelo SL no conjunto de teste...")
    y_pred_sl = lgbm_regressor.predict(df_test_sl_X)

    mae_sl = mean_absolute_error(y_test_sl, y_pred_sl)
    r2_sl = r2_score(y_test_sl, y_pred_sl)

    print(f"----------- MÉTRICAS DE PRECISÃO (Regressão do Lucro) -----------")
    print(f"Erro Médio Absoluto (MAE): ${mae_sl:,.2f}")
    print(f"R-quadrado (R²): {r2_sl:.2%}")
    print("----------------------------------------------------------------")
    print("Nota: Isto mede o quão bem o LGBM 'decorou' a função de simulação.")

else:
    print("PULANDO Etapa 9: Dataset 'df_rl_training' está vazio ou mapa de colunas não foi criado.")


--- Etapa 9: Iniciando Comparação com Supervised Learning (LGBM) ---
Formatando dados para o modelo de SL...
Dados SL divididos em 38520 para treino e 9630 para teste.

Treinando o modelo LightGBM para prever o lucro...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011862 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1294
[LightGBM] [Info] Number of data points in the train set: 38520, number of used features: 43
[LightGBM] [Info] Start training from score 9.997657
Modelo SL (Regressor) treinado com sucesso!

Avaliando a precisão do modelo SL no conjunto de teste...
----------- MÉTRICAS DE PRECISÃO (Regressão do Lucro) -----------
Erro Médio Absoluto (MAE): $0.01
R-quadrado (R²): 100.00%
----------------------------------------------------------------
N

In [ ]:
# --- Etapa 10: Salvando o Modelo e Componentes ---
print("\n--- Etapa 10: Salvando o Modelo e Componentes ---")

if agent_trained_successfully:
    try:
        cql_pricer.save_model('modelo_rl_churn_pricer.pt')
        print("Modelo salvo como 'modelo_rl_churn_pricer.pt'")

        with open('colunas_observacao_churn.json', 'w') as f:
            json.dump(observation_cols_map, f)
        print("Colunas de observação salvas como 'colunas_observacao_churn.json'")

        with open('config_tiers_churn.json', 'w') as f:
            json.dump(product_tiers, f)
        print("Configuração de Tiers salva como 'config_tiers_churn.json'")

    except Exception as e:
        print(f"Erro ao salvar arquivos: {e}")
else:
    print("Salvamento PULADO. Agente não foi treinado.")

print("\n--- FIM DO SCRIPT ---")


--- Etapa 10: Salvando o Modelo e Componentes ---
Modelo salvo como 'modelo_rl_churn_pricer.pt'
Colunas de observação salvas como 'colunas_observacao_churn.json'
Configuração de Tiers salva como 'config_tiers_churn.json'

--- FIM DO SCRIPT ---
